In [45]:
#Criação de um navegador e demais importações
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

nav = webdriver.Chrome()

In [46]:
#Importação e visualização da base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,2000,4000
1,rtx 3060,zota galax,4000,4500


In [47]:
def verificar_termos_banidos(lista_termos_banidos, nome):
    #Verificação se não há termos banidos no nome do produto
    tem_termos_banidos = False
    for termo in lista_termos_banidos:
        if termo in nome:
            tem_termos_banidos = True
    return tem_termos_banidos
def verificar_termos_requeridos(lista_termos_requeridos, nome):
    #Verificar se o nome do produto tem os termos requeridos
    tem_termos_requeridos = True
    for termo in lista_termos_requeridos:
        if termo not in nome:
            tem_termos_requeridos = False
    return tem_termos_requeridos


def busca_google_shopping(nav, termos_requeridos, termos_banidos, preco_minimo, preco_maximo):
    #Nesse dicionário ficarão os resultados filtrados
    dict_resultados = {}
    dict_resultados['nome'] = []
    dict_resultados['preco'] = []
    dict_resultados['link'] = []
    
    #tratamentodos parâmetros recebidos da tabela
    lista_termos_requeridos = termos_requeridos.lower().split(" ")
    lista_termos_banidos = termos_banidos.lower().split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #Procurar produto no Google Shopping
    #Entrar no Google
    nav.get("https://www.google.com/")

    #Pesquisar pelo produto
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(" ".join(lista_termos_requeridos), Keys.ENTER)

    #Entrar na aba shopping
    time.sleep(2)
    abas = nav.find_elements('class name', 'GKS7s')
    for item in abas:
        if "Shopping" in item.text:
            item.click()
            break

    #Pegar informações sobre o produto
    lista_resultados = nav.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'a8Pemb').text

        #Tratamento do dado do preço
        preco = float(preco.replace("R$","").replace("impostos","").replace("+","").replace(" ","").replace(".","").replace(",","."))

        nome = resultado.find_element('class name', 'tAxDx').text.lower()

        #Verificação se não há termos banidos no nome do produto
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)

        #Verificar se o nome do produto tem os termos requeridos
        tem_termos_requeridos = verificar_termos_requeridos(lista_termos_requeridos, nome)

        #Selecionar somente os elementos sem termos banidos e com todos os termos requeridos e se  está dentro da faixa de preços
        if tem_termos_banidos == False and tem_termos_requeridos==True:
            if preco_minimo <= preco <= preco_maximo:
                referencia_link = resultado.find_element('class name', 'bONr3b')
                elemento_pai = referencia_link.find_element('xpath','..')
                link = elemento_pai.get_attribute('href')
                dict_resultados['nome'].append(nome)
                dict_resultados['preco'].append(preco)
                dict_resultados['link'].append(link)
    return pd.DataFrame.from_dict(dict_resultados)

def busca_buscape(nav, termos_requeridos, termos_banidos, preco_minimo, preco_maximo):
    #Nesse dicionário ficarão os resultados filtrados
    dict_resultados = {}
    dict_resultados['nome'] = []
    dict_resultados['preco'] = []
    dict_resultados['link'] = []
    
    #tratamentodos parâmetros recebidos da tabela
    lista_termos_requeridos = termos_requeridos.lower().split(" ")
    lista_termos_banidos = termos_banidos.lower().split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #Procurar produto no Buscapé
    #Entrar no Google
    nav.get("https://www.buscape.com.br/")

    #Pesquisar pelo produto
    nav.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(" ".join(lista_termos_requeridos), Keys.ENTER)

    #Verificar carregamento da página
    while len(nav.find_elements('class name', 'Select_Select__1HNob')) < 1:
        time.sleep(1)

    #Pegar informações sobre o produto
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text

        #Tratamento do dado do preço
        preco = float(preco.replace("R$","").replace(" ","").replace(".","").replace(",","."))

        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text.lower()

        #Verificação se não há termos banidos no nome do produto
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)

        #Verificar se o nome do produto tem os termos requeridos
        tem_termos_requeridos = verificar_termos_requeridos(lista_termos_requeridos, nome)

        #Selecionar somente os elementos sem termos banidos e com todos os termos requeridos e se  está dentro da faixa de preços
        if tem_termos_banidos == False and tem_termos_requeridos == True:
            if preco_minimo <= preco <= preco_maximo:
                link = resultado.get_attribute('href')
                dict_resultados['nome'].append(nome)
                dict_resultados['preco'].append(preco)
                dict_resultados['link'].append(link)
    return pd.DataFrame.from_dict(dict_resultados)

In [48]:
tabela_ofertas = pd.DataFrame()
#Buscar atributos dos itens na tabela de produtos
for linha in tabela_produtos.index:    
    termos_requeridos = tabela_produtos.loc[linha,'Nome']
    termos_banidos = tabela_produtos.loc[linha,'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha,'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha,'Preço máximo']
    
    #Salvar as ofertas filtradas em um dataframe
    resultado_buscape = busca_buscape(nav, termos_requeridos, termos_banidos, preco_minimo, preco_maximo)
    if resultado_buscape.shape[0] > 0:
        tabela_ofertas = pd.concat([tabela_ofertas, resultado_buscape])
    
    resultado_google_shopping = busca_google_shopping(nav, termos_requeridos, termos_banidos, preco_minimo, preco_maximo)
    if resultado_google_shopping.shape[0] > 0:
        tabela_ofertas = pd.concat([tabela_ofertas, resultado_google_shopping])
        
display(tabela_ofertas)

,nome,preco,link
0,smartphone apple iphone 12 64gb câmera dupla,2479.00,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone 12 vermelho 64gb câmer...,3599.00,https://www.buscape.com.br/celular/smartphone-...
2,smartphone apple iphone 12 vermelho usado 64gb,2177.10,https://www.buscape.com.br/celular/smartphone-...
0,vitrine iphone 12 preto 64gb,2621.99,https://www.google.com/url?url=https://www.car...
1,vitrine iphone 12 branco 64gb,2621.99,https://www.google.com/url?url=https://www.car...
2,apple iphone 12 64gb 5g - vitrine - tela super...,2183.54,https://www.google.com/url?url=https://www.hor...
3,usado: iphone 12 64gb preto bom - trocafone,2339.00,https://www.google.com/url?url=https://www.cas...
4,"vitrine iphone 12 64gb roxo tela 6,1 4g câmera...",2499.00,https://www.google.com/url?url=https://www.car...
5,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",2499.00,https://www.google.com/url?url=https://www.car...
6,usado: iphone 12 64gb preto excelente - trocafone,2709.00,https://www.google.com/url?url=https://www.cas...


In [49]:
#Exportar para o Excel
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

In [50]:
#Enviar por e-mail o resultado da tabela
import win32com.client as win32

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'celiojr@hotmail.com'
    mail.Subject = 'Produtos encontrados na faixa de preço solicitada'
    mail.HTMLBody = f"""
    <p>Prezados</p>
    <p>Foram encontrados os seguintes produtos dentro da faiza de preços especificadas:
    {tabela_ofertas.to_html(index=False)}
    """
    mail.Send()
nav.quit()